<a href="https://colab.research.google.com/github/victwise/DeepLearning/blob/master/Cap%C3%ADtulo7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Psutil es una excelente manera de verificar el uso de la memoria. Esto será útil aquí ya que estamos utilizando un conjunto de datos más grande.

In [0]:
import psutil
import os

In [0]:
process = psutil.Process(os.getpid())
t = process.memory_info()

In [6]:
t.vms, t.rss

(730591232, 139575296)

In [0]:
def mem_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / psutil.virtual_memory().total

In [8]:
mem_usage()

0.010227358965121815

2. TQDM te da barras de progreso.

In [0]:
from time import sleep

In [10]:
# Without TQDM
s = 0
for i in range(10):
    s += i
    sleep(0.2)
print(s)

45


In [11]:
# With TQDM
from tqdm import tqdm

s = 0
for i in tqdm(range(10)):
    s += i
    sleep(0.2)
print(s)

100%|██████████| 10/10 [00:02<00:00,  4.96it/s]

45


##Aplicación SVD adicional 

Un uso interesante de la SVD que encontré recientemente fue como un paso en la eliminación de sesgos de incrustaciones de palabras de Word2Vec, desde Cuantificar y reducir estereotipos en incrustaciones de palabras (Bolukbasi, et al).

Word2Vec es una biblioteca útil lanzada por Google que representa palabras como vectores. La similitud de los vectores captura el significado semántico, y se pueden encontrar analogías, como París: Francia :: Tokio: Japón .

(fuente: Representaciones vectoriales de palabras )

Sin embargo, estas incrustaciones pueden codificar implícitamente sesgos, como padre: médico :: madre: enfermera y hombre: programador de computadoras :: mujer: ama de casa .

Un enfoque para eliminar la desviación del espacio implica el uso de SVD para reducir la dimensionalidad ( artículo de Bolukbasi ).

##Maneras de pensar en SVD 

- Compresión de datos
- SVD intercambia una gran cantidad de funciones por un conjunto más pequeño de características mejores
- Todas las matrices son diagonales (si utiliza el cambio de bases en el dominio y rango)

##Eigen Descomposición 

Los mejores métodos clásicos para calcular la SVD son las variantes de los métodos para calcular valores propios. Además de sus enlaces a SVD, las descomposiciones de Eigen también son útiles por sí mismas. Aquí hay algunas aplicaciones prácticas de descomposición eigen:

- poderes de matriz rápidos
- nth fibonacci numero
- Comportamiento de las EDOs
- Cadenas de Markov (economía del cuidado de la salud, Page Rank)
- Análisis Lineal Discriminante en el conjunto de datos Iris

##DBpedia Dataset 

In [0]:
import os, numpy as np, pickle
from bz2 import BZ2File
from datetime import datetime
from pprint import pprint
from time import time
from tqdm import tqdm_notebook
from scipy import sparse

from sklearn.decomposition import randomized_svd
from sklearn.externals.joblib import Memory
from urllib.request import urlopen

In [16]:
PATH = '/'
URL_BASE = 'http://downloads.dbpedia.org/3.5.1/en/'
filenames = ["redirects_en.nt.bz2", "page_links_en.nt.bz2"]

for filename in filenames:
    if not os.path.exists(PATH+filename):
        print("Downloading '%s', please wait..." % filename)
        open(PATH+filename, 'wb').write(urlopen(URL_BASE+filename).read())

In [0]:
redirects_filename = PATH+filenames[0]
page_links_filename = PATH+filenames[1]

In [0]:
DBPEDIA_RESOURCE_PREFIX_LEN = len("http://dbpedia.org/resource/")
SLICE = slice(DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)

In [0]:
def get_lines(filename): return (line.split() for line in BZ2File(filename))

In [0]:
def get_redirect(targ, redirects):
    seen = set()
    while True:
        transitive_targ = targ
        targ = redirects.get(targ)
        if targ is None or targ in seen: break
        seen.add(targ)
    return transitive_targ

In [0]:
def get_redirects(redirects_filename):
    redirects={}
    lines = get_lines(redirects_filename)
    return {src[SLICE]:get_redirect(targ[SLICE], redirects) 
                for src,_,targ,_ in tqdm_notebook(lines, leave=False)}

In [0]:
redirects = get_redirects(redirects_filename)

In [23]:
mem_usage()

0.1394518512341611

In [0]:
def add_item(lst, redirects, index_map, item):
    k = item[SLICE]
    lst.append(index_map.setdefault(redirects.get(k, k), len(index_map)))

In [0]:
limit=119077682 #5000000

In [0]:
# Computing the integer index map
index_map = dict() # links->IDs
lines = get_lines(page_links_filename)
source, destination, data = [],[],[]
for l, split in tqdm_notebook(enumerate(lines), total=limit):
    if l >= limit: break
    add_item(source, redirects, index_map, split[0])
    add_item(destination, redirects, index_map, split[2])
    data.append(1)

In [0]:
n=len(data); n